# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import json
import csv
import datetime
import time
from citipy import citipy
import random
from scipy.stats import linregress

# Google API Key
from api_keys import g_key

## Generate Cities List

In [2]:
# Setup the ranges for latitude and longitude
latitude_range = (-90,90)
longatude_range = (-180,180)

# Create variables for data
cities = []

no_cities = 2000
for cntr in range(no_cities):
    latitude = random.uniform(latitude_range[0],latitude_range[1])
    longitude = random.uniform(longatude_range[0],longatude_range[1])
    
    # If we have chosen a latitude of 0 then we don't want to use that
    # one
    if latitude != 0:
        city = citipy.nearest_city(latitude,longitude)
        city_name = city.city_name
        city_country = city.country_code
        if not any(c['name'] == city_name and c['country'] == city_country for c in cities):
            city_dict = {'name': city_name,'country': city_country}
            cities.append(city_dict)

In [3]:
print(cities)

[{'name': 'jinchang', 'country': 'cn'}, {'name': 'kapaa', 'country': 'us'}, {'name': 'mount isa', 'country': 'au'}, {'name': 'cocorit', 'country': 'mx'}, {'name': 'busselton', 'country': 'au'}, {'name': 'narsaq', 'country': 'gl'}, {'name': 'kloulklubed', 'country': 'pw'}, {'name': 'constitucion', 'country': 'mx'}, {'name': 'cidreira', 'country': 'br'}, {'name': 'bethel', 'country': 'us'}, {'name': 'temaraia', 'country': 'ki'}, {'name': 'cherskiy', 'country': 'ru'}, {'name': 'luderitz', 'country': 'na'}, {'name': 'punta arenas', 'country': 'cl'}, {'name': 'lolua', 'country': 'tv'}, {'name': 'ilam', 'country': 'np'}, {'name': 'tawkar', 'country': 'sd'}, {'name': 'taolanaro', 'country': 'mg'}, {'name': 'kalabugao', 'country': 'ph'}, {'name': 'tuktoyaktuk', 'country': 'ca'}, {'name': 'russell', 'country': 'nz'}, {'name': 'upernavik', 'country': 'gl'}, {'name': 'la ronge', 'country': 'ca'}, {'name': 'chapais', 'country': 'ca'}, {'name': 'rikitea', 'country': 'pf'}, {'name': 'salinas', 'coun

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
base_url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
city_id = []
city_name = []
city_cloudiness = []
city_country = []
city_date = []
city_humidity = []
city_latitude = []
city_longitude = []
city_temperature = []
city_wind_speed = []


# Build partial query URL
query_url = f"{base_url}appid={g_key}&units={units}&q="

city_cntr = -1
no_cities_to_retrieve = 525


# Loop through cities
for index, city in enumerate(cities):
    # See if we need to sleep so we don't go over our 60 a minute max for the api
    q, mod = divmod(index+1,55)
    if mod == 0:
        print("Pausing for a minute so we don't max out our api per minutes limit!")
        time.sleep(60)
    
    
    
    # Retrieve data information
    try:
        city_json = requests.get(query_url + city['name'] + ', ' + city['country']).json()
    except:
        print(f"City not found - {city['name']}, {city['country']} - skipping")

    if city_json['cod'] == 200:
        city_cntr += 1
        print(f"Retrieved data for {city_cntr}-{city_json['name']}, {city_json['sys']['country']} ({city_json['coord']['lat']},{city_json['coord']['lon']})")
        
        city_id.append(city_cntr)
        city_name.append(city_json['name'])
        city_cloudiness.append(city_json['clouds']['all'])
        city_country.append(city_json['sys']['country'])
        city_date.append(city_json['dt'])
        city_humidity.append(city_json['main']['humidity'])
        city_latitude.append(city_json['coord']['lat'])
        city_longitude.append(city_json['coord']['lon'])
        city_temperature.append(city_json['main']['temp_max'])              
        city_wind_speed.append(city_json['wind']['speed'])
    
    if city_cntr+1 == no_cities_to_retrieve:
        break
    
          

Retrieved data for 0-Jinchang, CN (38.5,102.17)
Retrieved data for 1-Kapaa, US (22.08,-159.32)
Retrieved data for 2-Mount Isa, AU (-20.73,139.5)
Retrieved data for 3-Cocorit, MX (27.58,-109.97)
Retrieved data for 4-Busselton, AU (-33.65,115.33)
Retrieved data for 5-Narsaq, GL (60.92,-46.05)
Retrieved data for 6-Kloulklubed, PW (7.04,134.26)
Retrieved data for 7-Constitucion, MX (23.99,-104.67)
Retrieved data for 8-Cidreira, BR (-30.18,-50.21)
Retrieved data for 9-Bethel, US (41.37,-73.41)
Retrieved data for 10-Cherskiy, RU (68.75,161.3)
Retrieved data for 11-Lüderitz, NA (-26.65,15.16)
Retrieved data for 12-Punta Arenas, CL (-53.15,-70.92)
Retrieved data for 13-Ilām, NP (26.92,87.92)
Retrieved data for 14-Tuktoyaktuk, CA (69.45,-133.04)
Retrieved data for 15-Russell, NZ (-35.26,174.12)
Retrieved data for 16-Upernavik, GL (72.79,-56.15)
Retrieved data for 17-La Ronge, CA (55.1,-105.28)
Retrieved data for 18-Chapais, CA (49.78,-74.85)
Retrieved data for 19-Rikitea, PF (-23.12,-134.97)
Re

Retrieved data for 161-Cabo San Lucas, MX (22.89,-109.91)
Retrieved data for 162-Vostok, RU (46.49,135.88)
Retrieved data for 163-San Bartolomé de Tirajana, ES (27.92,-15.57)
Retrieved data for 164-Garissa, KE (-0.45,39.64)
Retrieved data for 165-Iqaluit, CA (63.75,-68.51)
Retrieved data for 166-Comodoro Rivadavia, AR (-45.87,-67.5)
Retrieved data for 167-Olafsvik, IS (64.89,-23.71)
Retrieved data for 168-Juelsminde, DK (55.71,10.02)
Retrieved data for 169-Toppenish, US (46.38,-120.31)
Retrieved data for 170-Tazovsky, RU (67.47,78.7)
Retrieved data for 171-Asunción Nochixtlán, MX (17.47,-97.23)
Retrieved data for 172-Bar Harbor, US (44.39,-68.2)
Retrieved data for 173-Okhotsk, RU (59.38,143.3)
Retrieved data for 174-Chokurdakh, RU (70.63,147.92)
Retrieved data for 175-Puerto Ayora, EC (-0.74,-90.35)
Retrieved data for 176-Bullhead City, US (35.15,-114.57)
Retrieved data for 177-Buldāna, IN (20.53,76.18)
Retrieved data for 178-Griffith, AU (-34.28,146.03)
Retrieved data for 179-Slave La

Retrieved data for 319-Seoul, KR (37.57,126.98)
Retrieved data for 320-Key Largo, US (25.09,-80.45)
Retrieved data for 321-Pilar, PH (14.66,120.57)
Retrieved data for 322-Laurel, US (39.1,-76.85)
Retrieved data for 323-Zeya, RU (53.75,127.27)
Retrieved data for 324-Neryungri, RU (56.67,124.65)
Retrieved data for 325-Rexburg, US (43.83,-111.79)
Retrieved data for 326-Brae, GB (60.4,-1.35)
Retrieved data for 327-Weiz, AT (47.22,15.62)
Retrieved data for 328-Vanavara, RU (60.34,102.28)
Retrieved data for 329-Aripuanã, BR (-9.17,-60.63)
Retrieved data for 330-Tura, RU (64.28,100.25)
Retrieved data for 331-Kokopo, PG (-4.35,152.26)
Retrieved data for 332-Shāhpur, IN (16.7,76.83)
Retrieved data for 333-Kizema, RU (61.11,44.83)
Retrieved data for 334-Siilinjärvi, FI (63.08,27.67)
Retrieved data for 335-Mao, TD (14.12,15.31)
Retrieved data for 336-Acaraú, BR (-2.89,-40.12)
Retrieved data for 337-Suzun, RU (53.78,82.31)
Retrieved data for 338-Bandarbeyla, SO (9.49,50.81)
Retrieved data for 339-

Retrieved data for 478-Virginia Beach, US (36.85,-75.98)
Retrieved data for 479-Suba, PH (10.84,121.01)
Retrieved data for 480-Baracoa, CU (20.35,-74.5)
Retrieved data for 481-Tebingtinggi, ID (3.33,99.16)
Retrieved data for 482-Dingzhou, CN (38.51,115)
Retrieved data for 483-Sharlyk, RU (52.92,54.75)
Retrieved data for 484-Seymchan, RU (62.88,152.43)
Retrieved data for 485-Grimshaw, CA (56.18,-117.6)
Retrieved data for 486-Montorio al Vomano, IT (42.58,13.64)
Retrieved data for 487-Bilibino, RU (68.05,166.44)
Retrieved data for 488-Adrar, DZ (25.75,-1)
Retrieved data for 489-Sehwan, PK (26.43,67.86)
Retrieved data for 490-San Rafael, AR (-34.62,-68.33)
Retrieved data for 491-Hundested, DK (55.97,11.87)
Pausing for a minute so we don't max out our api per minutes limit!
Retrieved data for 492-Dawson Creek, CA (55.77,-120.24)
Retrieved data for 493-Kabanjahe, ID (3.1,98.49)
Retrieved data for 494-São José da Coroa Grande, BR (-8.9,-35.15)
Retrieved data for 495-Rieti, IT (42.41,12.86)
R

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [5]:
# Create a DataFrame with the following columns:
# City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
city_data = {
             "City ID": city_id,
             "City": city_name,
             "Cloudiness (%)": city_cloudiness,
             "Country": city_country,    
             "Date": city_date,
             "Humidity (%)": city_humidity,
             "Lat": city_latitude,
             "Lng": city_longitude,
             "Max Temp": city_temperature,
             "Wind Speed (mph)": city_wind_speed
            }

city_df = pd.DataFrame(city_data)
city_df.set_index('City ID', inplace = True)

# Write the cities.csv file
csv_file_name = "../output_data/cities.csv"

city_df.to_csv(csv_file_name)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [6]:
# Get rid of any cities that have a humitidy greater than 100
cleaned_city_df = city_df[city_df['Humidity (%)'] <= 100]

# Preview the cleaned data
cleaned_city_df

,City,Cloudiness (%),Country,Date,Humidity (%),Lat,Lng,Max Temp,Wind Speed (mph)
City ID,,,,,,,,,
0,Jinchang,0,CN,1603041137,50,38.50,102.17,45.45,4.45
1,Kapaa,75,US,1603040802,94,22.08,-159.32,77.00,8.05
2,Mount Isa,25,AU,1603040545,25,-20.73,139.50,84.20,8.05
3,Cocorit,5,MX,1603041137,70,27.58,-109.97,84.20,3.36
4,Busselton,2,AU,1603040802,49,-33.65,115.33,62.01,10.45
...,...,...,...,...,...,...,...,...,...
520,Clyde River,20,CA,1603041581,85,70.47,-68.59,21.20,4.70
521,Sisimiut,100,GL,1603041778,65,66.94,-53.67,42.64,17.78
522,Yulara,90,AU,1603041380,88,-25.24,130.99,68.00,13.87


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression